In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/football-match-probability-prediction/sample_submission.csv
/kaggle/input/football-match-probability-prediction/train.csv
/kaggle/input/football-match-probability-prediction/test.csv


In [2]:
train = pd.read_csv("../input/football-match-probability-prediction/train.csv", dtype=object)
#../input/football-match-probability-prediction/train.csv

### Start by looking at what sort of features we do have in the provided training dataset:

In [3]:
print(len(list(train.columns)),list(train.columns))

190 ['id', 'target', 'home_team_name', 'away_team_name', 'match_date', 'league_name', 'league_id', 'is_cup', 'home_team_coach_id', 'away_team_coach_id', 'home_team_history_match_date_1', 'home_team_history_match_date_2', 'home_team_history_match_date_3', 'home_team_history_match_date_4', 'home_team_history_match_date_5', 'home_team_history_match_date_6', 'home_team_history_match_date_7', 'home_team_history_match_date_8', 'home_team_history_match_date_9', 'home_team_history_match_date_10', 'home_team_history_is_play_home_1', 'home_team_history_is_play_home_2', 'home_team_history_is_play_home_3', 'home_team_history_is_play_home_4', 'home_team_history_is_play_home_5', 'home_team_history_is_play_home_6', 'home_team_history_is_play_home_7', 'home_team_history_is_play_home_8', 'home_team_history_is_play_home_9', 'home_team_history_is_play_home_10', 'home_team_history_is_cup_1', 'home_team_history_is_cup_2', 'home_team_history_is_cup_3', 'home_team_history_is_cup_4', 'home_team_history_is_cup

### One by one, we'll look at features and try to understand what potential they have:

**Match Dates**

In [4]:
column_name_list = ["home_team_history_match_date_" + str(i) for i in np.arange(1,11,1)]
column_name_list.insert(0, "match_date")
train[column_name_list][train.home_team_name =="Arsenal"].head(1)

,match_date,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10
1182,2019-12-05 20:15:00,2019-12-01 14:00:00,2019-11-28 20:00:00,2019-11-23 15:00:00,2019-11-09 17:30:00,2019-11-06 15:50:00,2019-11-02 15:00:00,2019-10-30 19:30:00,2019-10-27 16:30:00,2019-10-24 19:00:00,2019-10-21 19:00:00


It seems like we can decide:
- whether it is a night fixture or day fixture,
- a winter or summer game,
- fatigue, congestion and inherent injury information by looking at the tightness of the last 10 fixtures
-----

****